# 제주 빅데이터 경진대회

2020.07.27 월요일 김채형

**랜덤포레스트 하이퍼파라미터 튜닝**

In [5]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import joblib

In [7]:
data = pd.read_csv('C:/Users/dhxog/Desktop/데이콘/dacon_competition/april_add_merge_with_homeproportion.csv', encoding='EUC-KR', index_col='Unnamed: 0')

C:\Users\dhxog\anaconda3\envs\my\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [8]:
data

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,festival_count,year,month,rain_day,total_tourist,corona_confirmed,total_sale_by_indst,sex_age_population,total_home
1,강원,건강보조식품 소매업,강원,20s,1,1,3,27500.0,3,8,2019,8,13.9,1603895,0,33849.0,100115,0.57
2,강원,건강보조식품 소매업,강원,20s,1,1,3,139000.0,3,8,2019,6,10.6,1374767,0,33849.0,100115,0.72
3,강원,건강보조식품 소매업,강원,20s,1,1,3,395500.0,3,11,2019,9,9.8,2062151,0,33849.0,100115,0.72
4,강원,건강보조식품 소매업,강원,20s,1,1,3,427510.0,2,0,2020,3,7.3,761521,29,33849.0,100115,0.82
5,강원,건강보조식품 소매업,강원,20s,1,1,3,605000.0,3,4,2019,2,4.7,2146566,0,33849.0,100115,0.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114447,충북,휴양콘도 운영업,충북,70s,2,5,3,148000.0,4,0,2019,3,7.3,144712,0,50447.0,65436,0.83
1114448,충북,휴양콘도 운영업,충북,70s,2,5,3,230400.0,3,0,2020,1,6.9,253247,0,50447.0,65436,0.83
1114449,충북,휴양콘도 운영업,충북,70s,2,5,3,247800.0,3,0,2019,12,9.1,156772,0,50447.0,65436,0.84
1114450,충북,휴양콘도 운영업,충북,70s,2,5,5,329800.0,7,0,2019,5,7.3,144712,0,50447.0,65436,0.81


In [9]:
data.shape

(1114451, 18)

### 데이터 전처리

In [10]:
data = data.drop(columns=['CNT', 'CSTMR_CNT'])

In [11]:
# 데이터 정제
data = data.fillna('')
df = data.copy()
columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [12]:
df_num = df.copy()

In [13]:
cat_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE']
df_num[cat_features]= df_num[cat_features].apply(LabelEncoder().fit_transform)

In [14]:
real_cat_features =  ['SEX_CTGO_CD','CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE','FLC']
for i in enumerate(real_cat_features):
    ca = i[1]
    df_num[ca] = df_num[ca].astype('category')

### 모델링

In [15]:
X = df_num[df_num.columns.difference(['AMT'])]
y = df_num[['AMT']]

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    random_state=2020)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = { 'n_estimators' : [10, 100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }

# GridSearchCV 수행
rf_reg = RandomForestRegressor(random_state = 2020, n_jobs = -1)
rf_grid = GridSearchCV(rf_reg, param_grid = params, cv = 3, n_jobs = -1)
rf_grid.fit(X_train, np.ravel(y_train))

In [ ]:
rf_grid.best_params_

---- 쓰레기 ----

In [ ]:
from sklearn.model_selection import train_test_split
X = data.drop(columns=['AMT'])
y = data['AMT']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False,random_state=2020)
X_train.shape

In [ ]:
X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
X_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_test.columns]

In [ ]:
from sklearn.model_selection import GridSearchCV

params = { 'n_estimators' : [10, 100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }

# GridSearchCV 수행
rf_reg = RandomForestRegressor(random_state = 2020, n_jobs = -1)
rf_grid = GridSearchCV(rf_reg, param_grid = params, cv = 3, n_jobs = -1)
rf_grid.fit(X_train, y_train)

In [ ]:
rf_grid.best_params_

In [ ]:
# 데이터 정제
data = data.fillna('')
df = data.copy()
columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [ ]:
df_num = df.copy()

In [ ]:
cat_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE']
df_num[cat_features]= df_num[cat_features].apply(LabelEncoder().fit_transform)

In [ ]:
real_cat_features =  ['SEX_CTGO_CD','CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE','FLC']
for i in enumerate(real_cat_features):
    ca = i[1]
    df_num[ca] = df_num[ca].astype('category')

### 모델링

In [ ]:
X = df_num[df_num.columns.difference(['AMT'])]
y = df_num[['AMT']]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    random_state=2020)

In [ ]:
model = joblib.load('rf_initial.pkl')
model

In [ ]:
from sklearn.model_selection import GridSearchCV

params = { 'n_estimators' : [10, 100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
# rf_clf = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(model, param_grid = params, cv = 3, n_jobs = -1)
grid_cv.fit(X_train, y_train)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

In [ ]:
from sklearn.model_selection import GridSearchCV
 
param_grid = [
        {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
        {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
    ]
rf_reg = RandomForestRegressor()
 
grid_search = GridSearchCV(rf_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
 
grid_search.fit(X_train, np.ravel(y_train))